# Signal Combinations Framework

Compact implementation of multi-signal trading strategies:
- **Regime-Aware**: Adapt to market conditions
- **Confirmation**: Multiple signals must agree  
- **Ensemble**: Weighted voting systems
- **Hierarchical**: Filtered approaches

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

try:
    from AlgorithmImports import *
    qb = QuantBook()
except:
    qb = None

np.random.seed(42)
print("Signal Combinations Framework Ready")

In [13]:
class SignalCombinations:
    """Compact signal combination strategies"""
    
    def __init__(self, signals_dict, data):
        self.signals = signals_dict
        self.data = data
        self.combinations = {}
    
    def _get_signals(self, signal_names):
        """Get signals by name, return empty list if not found"""
        return [self.signals[name] for name in signal_names if name in self.signals]
    
    def _consensus(self, signals, threshold=0.5):
        """Calculate consensus from multiple signals"""
        if not signals: return np.zeros(len(self.data))
        votes = np.mean(signals, axis=0)
        return np.where(votes > threshold, 1, np.where(votes < -threshold, -1, 0))
    
    def volatility_regime(self):
        """Use momentum in high vol, mean reversion in low vol"""
        returns = self.data['close'].pct_change()
        vol_regime = returns.rolling(20).std() > returns.rolling(100).std()
        
        momentum_sigs = self._get_signals(['Fractal_Hurst', 'KAMA', 'Donchian_Breakout'])
        mean_rev_sigs = self._get_signals(['VWAP_ZScore', 'BB_Squeeze', 'Gap_Fill'])
        
        combined = np.zeros(len(self.data))
        if momentum_sigs and mean_rev_sigs:
            momentum_consensus = self._consensus(momentum_sigs, 0.3)
            mean_rev_consensus = self._consensus(mean_rev_sigs, 0.3)
            combined = np.where(vol_regime, momentum_consensus, mean_rev_consensus)
        
        self.combinations['Volatility_Regime'] = combined
        return combined
    
    def momentum_confirmation(self):
        """Require 70% agreement from momentum signals"""
        momentum_sigs = self._get_signals(['Fractal_Hurst', 'KAMA', 'Donchian_Breakout', 'ML_GradientBoosting'])
        combined = self._consensus(momentum_sigs, 0.7) if len(momentum_sigs) >= 3 else np.zeros(len(self.data))
        self.combinations['Momentum_Confirmation'] = combined
        return combined
    
    def ensemble_weighted(self):
        """Weight signals by performance"""
        returns = self.data['close'].pct_change().shift(-1)
        weights = {}
        
        # Calculate weights based on signal performance
        for name, signals in self.signals.items():
            if name == 'Random_Baseline': continue
            signal_returns = [signals[i] * returns.iloc[i] for i in range(len(signals)) if signals[i] != 0 and i < len(returns)-1]
            if len(signal_returns) > 10:
                sharpe = np.mean(signal_returns) / np.std(signal_returns) if np.std(signal_returns) > 0 else 0
                weights[name] = max(0, sharpe)
        
        # Normalize weights
        total_weight = sum(weights.values())
        if total_weight > 0:
            weights = {k: v/total_weight for k, v in weights.items()}
        
        # Combine with weights
        combined = np.zeros(len(self.data))
        for i in range(len(self.data)):
            weighted_vote = sum(weights.get(name, 0) * signals[i] for name, signals in self.signals.items())
            combined[i] = 1 if weighted_vote > 0.3 else -1 if weighted_vote < -0.3 else 0
        
        self.combinations['Ensemble_Weighted'] = combined
        return combined
    
    def hierarchical_filter(self):
        """Multi-level filtering approach"""
        # Primary signals
        primary_sigs = self._get_signals(['ML_GradientBoosting', 'RV_Regime'])
        if not primary_sigs: return np.zeros(len(self.data))
        
        primary_consensus = self._consensus(primary_sigs, 0.4)
        
        # Volume filter
        if 'volume' in self.data.columns:
            vol_ma = self.data['volume'].rolling(20).mean()
            low_volume = self.data['volume'] < vol_ma * 0.7
            primary_consensus = np.where(low_volume, 0, primary_consensus)
        
        self.combinations['Hierarchical_Filter'] = primary_consensus
        return primary_consensus
    
    def run_all_strategies(self):
        """Execute all combination strategies"""
        strategies = [
            self.volatility_regime,
            self.momentum_confirmation,
            self.ensemble_weighted,
            self.hierarchical_filter
        ]
        
        for strategy in strategies:
            try:
                strategy()
                print(f"{strategy.__name__} - Done")
            except Exception as e:
                print(f"{strategy.__name__} failed: {e}")
        
        # Add random baseline
        self.combinations['Random_Baseline'] = np.random.choice([-1, 0, 1], size=len(self.data), p=[0.3, 0.4, 0.3])
        
        return self.combinations

print("SignalCombinations class ready")

In [14]:
def analyze_performance(data, signals_dict):
    """Compact performance analysis"""
    returns = data['close'].pct_change().shift(-1)
    results = []
    
    for name, signals in signals_dict.items():
        signal_returns = np.where(signals != 0, signals * returns, 0)
        valid_returns = signal_returns[signals != 0]
        
        if len(valid_returns) > 5:
            total_return = np.sum(signal_returns)
            avg_return = np.mean(valid_returns)
            volatility = np.std(valid_returns)
            sharpe = avg_return / volatility if volatility > 0 else 0
            frequency = np.sum(signals != 0) / len(signals)
            
            results.append({
                'strategy': name,
                'sharpe': sharpe,
                'total_return': total_return,
                'frequency': frequency,
                'signals': np.sum(signals != 0)
            })
    
    df = pd.DataFrame(results).sort_values('sharpe', ascending=False)
    return df

print("Performance analyzer ready")

In [15]:
# Demo execution with sample data
def run_demo():
    """Run demonstration with sample data"""
    # Create sample data
    dates = pd.date_range('2023-01-01', '2024-01-01', freq='D')
    prices = 50000 * np.cumprod(1 + np.random.normal(0.001, 0.02, len(dates)))
    
    data = pd.DataFrame({
        'close': prices,
        'high': prices * (1 + np.abs(np.random.normal(0, 0.01, len(dates)))),
        'low': prices * (1 - np.abs(np.random.normal(0, 0.01, len(dates)))),
        'volume': np.random.lognormal(10, 0.5, len(dates))
    }, index=dates)
    
    # Create sample signals
    sample_signals = {
        'Fractal_Hurst': np.random.choice([-1, 0, 1], len(data), p=[0.2, 0.6, 0.2]),
        'KAMA': np.random.choice([-1, 0, 1], len(data), p=[0.25, 0.5, 0.25]),
        'Donchian_Breakout': np.random.choice([-1, 0, 1], len(data), p=[0.15, 0.7, 0.15]),
        'VWAP_ZScore': np.random.choice([-1, 0, 1], len(data), p=[0.2, 0.6, 0.2]),
        'BB_Squeeze': np.random.choice([-1, 0, 1], len(data), p=[0.1, 0.8, 0.1]),
        'ML_GradientBoosting': np.random.choice([-1, 0, 1], len(data), p=[0.2, 0.6, 0.2]),
        'RV_Regime': np.random.choice([-1, 0, 1], len(data), p=[0.3, 0.4, 0.3]),
        'Gap_Fill': np.random.choice([-1, 0, 1], len(data), p=[0.1, 0.8, 0.1])
    }
    
    # Run combinations
    combiner = SignalCombinations(sample_signals, data)
    all_combinations = combiner.run_all_strategies()
    
    # Analyze performance
    performance = analyze_performance(data, all_combinations)
    
    print(f"\nResults for {len(all_combinations)} strategies:")
    print(performance.head(10).to_string(index=False))
    
    return data, all_combinations, performance

# Run demo if executed
if __name__ == "__main__":
    data, combinations, results = run_demo()
    print("\nDemo completed successfully!")

## Usage Instructions

### Quick Start:
```python
# Initialize with your signals and data
combiner = SignalCombinations(your_signals_dict, your_data)
results = combiner.run_all_strategies()

# Analyze performance
performance = analyze_performance(your_data, results)
print(performance)
```

### Strategies Included:
1. **Volatility Regime**: Momentum in high vol, mean reversion in low vol
2. **Momentum Confirmation**: 70% agreement threshold
3. **Ensemble Weighted**: Performance-based signal weighting
4. **Hierarchical Filter**: Multi-level filtering with volume confirmation

### Key Features:
- Compact implementation (~150 lines vs 1800+)
- Essential functionality preserved
- Easy to extend and modify
- Built-in performance analysis
- Sample data demo included